In [ ]:
pip install pymorphy2

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
import nltk
nltk.download('punkt')
import gensim
import string
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import files
uploaded =files.upload()

Saving train.xlsx to train (2).xlsx


In [ ]:
from google.colab import files
uploaded =files.upload()

Saving train.jsonl to train (2).jsonl
Saving val_wtihout.jsonl to val_wtihout (2).jsonl


In [ ]:
train_df=pd.read_excel("train.xlsx")

In [ ]:
import pandas as pd
import json
# функция, которая позволяет считать тренировочный набор данных и вернуть в виде Датафрейм
def get_X_y(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append(text+' Query: '+query+' Answer: '+a['text'])
                    y.append(a['label'])
    return X, y

X_train, y_train = get_X_y('train.jsonl')

train_df = pd.DataFrame({
    'text': X_train,
    'label': y_train
})

# в колонке 'text'
# лежит сам текст, вопрос, ответ на этот вопрос
# так как к одному тексту может быть несколько ответов, то похожих строк может быть несколько
train_df

,text,label
0,"(1) Но люди не могут существовать без природы,...",1
1,"(1) Но люди не могут существовать без природы,...",0
2,"(1) Но люди не могут существовать без природы,...",0
3,"(1) Но люди не могут существовать без природы,...",1
4,"(1) Но люди не могут существовать без природы,...",0
...,...,...
11945,(1) Известный американский боец смешанных един...,0
11946,(1) Известный американский боец смешанных един...,1
11947,(1) Известный американский боец смешанных един...,0
11948,(1) Известный американский боец смешанных един...,0


In [ ]:
train_df['text1']=""
train_df['Query']=""
train_df['Answer']=""
train_df

,text,label,text1,Query,Answer
0,"(1) Но люди не могут существовать без природы,...",1,,,
1,"(1) Но люди не могут существовать без природы,...",0,,,
2,"(1) Но люди не могут существовать без природы,...",0,,,
3,"(1) Но люди не могут существовать без природы,...",1,,,
4,"(1) Но люди не могут существовать без природы,...",0,,,
...,...,...,...,...,...
11945,(1) Известный американский боец смешанных един...,0,,,
11946,(1) Известный американский боец смешанных един...,1,,,
11947,(1) Известный американский боец смешанных един...,0,,,
11948,(1) Известный американский боец смешанных един...,0,,,


In [ ]:
for i in range(0,len(train_df)):
  a=train_df.text[i]
  b=a.find("Query")
  b1=a.find("Answer")
  text1=a[:b]
  train_df['text1'][i]=text1
  query=a[b+7:b1]
  train_df['Query'][i]=query
  ans=a[b1+8:]
  train_df['Answer'][i]=ans
train_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,text,label,text1,Query,Answer
0,"(1) Но люди не могут существовать без природы,...",1,"(1) Но люди не могут существовать без природы,...",Где бегала шпана?,В парке.
1,"(1) Но люди не могут существовать без природы,...",0,"(1) Но люди не могут существовать без природы,...",Где бегала шпана?,В лесу.
2,"(1) Но люди не могут существовать без природы,...",0,"(1) Но люди не могут существовать без природы,...",Где бегала шпана?,Около подъезда.
3,"(1) Но люди не могут существовать без природы,...",1,"(1) Но люди не могут существовать без природы,...",Почему Люда ударила Артема?,Он к ней приставал.
4,"(1) Но люди не могут существовать без природы,...",0,"(1) Но люди не могут существовать без природы,...",Почему Люда ударила Артема?,Он ее оскорбил.
...,...,...,...,...,...
11945,(1) Известный американский боец смешанных един...,0,(1) Известный американский боец смешанных един...,Как проявляется любовь известного американског...,Он сочинил гимн Советского Союза.
11946,(1) Известный американский боец смешанных един...,1,(1) Известный американский боец смешанных един...,Как проявляется любовь известного американског...,У него тату на русском языке и русская девушка...
11947,(1) Известный американский боец смешанных един...,0,(1) Известный американский боец смешанных един...,Как проявляется любовь известного американског...,На его теле есть татуировка с российским паспо...
11948,(1) Известный американский боец смешанных един...,0,(1) Известный американский боец смешанных един...,Как проявляется любовь известного американског...,Он пишет стихи и прозу на русском языке и даёт...


In [ ]:
import pandas as pd
import json
# функция которая позволяет считать данные из теста и записать их в Датафрейм
def get_X_test(data_json_file):
    X1 = []
    X2 = []
    X3 = []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X1.append(text) #текст
                    X2.append(query) #вопрос
                    X3.append(a['text']) #ответ на вопрос
    return X1, X2, X3

X1, X2, X3 = get_X_test('val_wtihout.jsonl')

test_df = pd.DataFrame({
    'text': X1,
    'query': X2,
    'answer': X3
})

test_df
# В колонке text лежит: текст, вопрос и ответ на этот вопрос
# Вам нужно реализовать алгоритм, который вернет, ответ на вопрос верный или нет

,text,query,answer
0,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,Так перевозили особо важных заключенных.
1,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что был эмигрантом."
2,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он сам вырыл себе землянку."
3,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он побывал на пересылке Красная Пр..."
4,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он был особо важным заключённым и ..."
...,...,...,...
2230,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Изнасилование.
2231,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко похитил любовника и присвоил его деньги.
2232,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Убийство и расчленение тела.
2233,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко убил любовника и расчленил его тело.


# Новый раздел

In [ ]:
morph = MorphAnalyzer()
qwes=[]
zap=[]
i=0
def lemmatize(doc):
    global qwes,zap,morph,i

    print(i,"/11950")

    
    i=i+1
    
    doc = re.sub(r'[^А-Яа-яё ]', " ", doc)
    for k in range (0,3):
      doc=doc.replace("  "," ")
    doc=doc[1:]
    doc=doc.split(" ")
    tokens=[]
    for token in doc:
      token = morph.normal_forms(token)
      tokens.append(token[0])
    return(tokens)

In [ ]:
train_df.text=train_df.text.apply(lemmatize)

Выходные данные были обрезаны до нескольких последних строк (5000).
6950 /11950
6951 /11950
6952 /11950
6953 /11950
6954 /11950
6955 /11950
6956 /11950
6957 /11950
6958 /11950
6959 /11950
6960 /11950
6961 /11950
6962 /11950
6963 /11950
6964 /11950
6965 /11950
6966 /11950
6967 /11950
6968 /11950
6969 /11950
6970 /11950
6971 /11950
6972 /11950
6973 /11950
6974 /11950
6975 /11950
6976 /11950
6977 /11950
6978 /11950
6979 /11950
6980 /11950
6981 /11950
6982 /11950
6983 /11950
6984 /11950
6985 /11950
6986 /11950
6987 /11950
6988 /11950
6989 /11950
6990 /11950
6991 /11950
6992 /11950
6993 /11950
6994 /11950
6995 /11950
6996 /11950
6997 /11950
6998 /11950
6999 /11950
7000 /11950
7001 /11950
7002 /11950
7003 /11950
7004 /11950
7005 /11950
7006 /11950
7007 /11950
7008 /11950
7009 /11950
7010 /11950
7011 /11950
7012 /11950
7013 /11950
7014 /11950
7015 /11950
7016 /11950
7017 /11950
7018 /11950
7019 /11950
7020 /11950
7021 /11950
7022 /11950
7023 /11950
7024 /11950
7025 /11950
7026 /11950
7027 /11

In [ ]:
train_df

,Unnamed: 0,text,label,text1,Query,Answer
0,0,"['но', 'человек', 'не', 'мочь', 'существовать'...",1,"['но', 'человек', 'не', 'мочь', 'существовать'...","['де', 'бегать', 'шпана', '']","['', 'парка', '']"
1,1,"['но', 'человек', 'не', 'мочь', 'существовать'...",0,"['но', 'человек', 'не', 'мочь', 'существовать'...","['де', 'бегать', 'шпана', '']","['', 'лес', '']"
2,2,"['но', 'человек', 'не', 'мочь', 'существовать'...",0,"['но', 'человек', 'не', 'мочь', 'существовать'...","['де', 'бегать', 'шпана', '']","['коло', 'подъезд', '']"
3,3,"['но', 'человек', 'не', 'мочь', 'существовать'...",1,"['но', 'человек', 'не', 'мочь', 'существовать'...","['очий', 'люд', 'ударить', 'артём', '']","['н', 'к', 'она', 'приставать', '']"
4,4,"['но', 'человек', 'не', 'мочь', 'существовать'...",0,"['но', 'человек', 'не', 'мочь', 'существовать'...","['очий', 'люд', 'ударить', 'артём', '']","['н', 'она', 'оскорбить', '']"
...,...,...,...,...,...,...
11945,11945,"['известный', 'американский', 'боец', 'смешанн...",0,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['н', 'сочинить', 'гимн', 'советский', 'союз',..."
11946,11946,"['известный', 'американский', 'боец', 'смешанн...",1,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['', 'он', 'тат', 'на', 'русский', 'язык', 'и'..."
11947,11947,"['известный', 'американский', 'боец', 'смешанн...",0,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['а', 'он', 'тело', 'есть', 'татуировка', 'с',..."
11948,11948,"['известный', 'американский', 'боец', 'смешанн...",0,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['н', 'писать', 'стих', 'и', 'проза', 'на', 'р..."


In [ ]:
for i in range(0,2235):
  test_df.text[i]= test_df['text'][i]+ test_df['query'][i]+ test_df['answer'][i]

In [ ]:
i=0
test_df['text']=test_df['text'].apply(lemmatize)

0 /11950
1 /11950
2 /11950
3 /11950
4 /11950
5 /11950
6 /11950
7 /11950
8 /11950
9 /11950
10 /11950
11 /11950
12 /11950
13 /11950
14 /11950
15 /11950
16 /11950
17 /11950
18 /11950
19 /11950
20 /11950
21 /11950
22 /11950
23 /11950
24 /11950
25 /11950
26 /11950
27 /11950
28 /11950
29 /11950
30 /11950
31 /11950
32 /11950
33 /11950
34 /11950
35 /11950
36 /11950
37 /11950
38 /11950
39 /11950
40 /11950
41 /11950
42 /11950
43 /11950
44 /11950
45 /11950
46 /11950
47 /11950
48 /11950
49 /11950
50 /11950
51 /11950
52 /11950
53 /11950
54 /11950
55 /11950
56 /11950
57 /11950
58 /11950
59 /11950
60 /11950
61 /11950
62 /11950
63 /11950
64 /11950
65 /11950
66 /11950
67 /11950
68 /11950
69 /11950
70 /11950
71 /11950
72 /11950
73 /11950
74 /11950
75 /11950
76 /11950
77 /11950
78 /11950
79 /11950
80 /11950
81 /11950
82 /11950
83 /11950
84 /11950
85 /11950
86 /11950
87 /11950
88 /11950
89 /11950
90 /11950
91 /11950
92 /11950
93 /11950
94 /11950
95 /11950
96 /11950
97 /11950
98 /11950
99 /11950
100 /11950

In [ ]:
test_df

,text,query,answer
0,"[самый, первый, остров, архипелаг, возникнуть,...",Почему Солженицына перевозили спецконвоем?,Так перевозили особо важных заключенных.
1,"[самый, первый, остров, архипелаг, возникнуть,...",Почему Солженицына перевозили спецконвоем?,"Потому, что был эмигрантом."
2,"[самый, первый, остров, архипелаг, возникнуть,...",Почему Солженицына перевозили спецконвоем?,"Потому, что он сам вырыл себе землянку."
3,"[самый, первый, остров, архипелаг, возникнуть,...",Почему Солженицына перевозили спецконвоем?,"Потому, что он побывал на пересылке Красная Пр..."
4,"[самый, первый, остров, архипелаг, возникнуть,...",Почему Солженицына перевозили спецконвоем?,"Потому, что он был особо важным заключённым и ..."
...,...,...,...
2230,"[канадский, порноактёр, лука, рокко, маньотт, ...",За что осуждён Рокко Маньотта?,Изнасилование.
2231,"[канадский, порноактёр, лука, рокко, маньотт, ...",За что осуждён Рокко Маньотта?,Рокко похитил любовника и присвоил его деньги.
2232,"[канадский, порноактёр, лука, рокко, маньотт, ...",За что осуждён Рокко Маньотта?,Убийство и расчленение тела.
2233,"[канадский, порноактёр, лука, рокко, маньотт, ...",За что осуждён Рокко Маньотта?,Рокко убил любовника и расчленил его тело.


In [ ]:
test_df.to_excel('test.xlsx')

In [ ]:
from google.colab import files
files.download("test.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train_df

,Unnamed: 0,text,label,text1,Query,Answer
0,0,"['но', 'человек', 'не', 'мочь', 'существовать'...",1,"['но', 'человек', 'не', 'мочь', 'существовать'...","['де', 'бегать', 'шпана', '']","['', 'парка', '']"
1,1,"['но', 'человек', 'не', 'мочь', 'существовать'...",0,"['но', 'человек', 'не', 'мочь', 'существовать'...","['де', 'бегать', 'шпана', '']","['', 'лес', '']"
2,2,"['но', 'человек', 'не', 'мочь', 'существовать'...",0,"['но', 'человек', 'не', 'мочь', 'существовать'...","['де', 'бегать', 'шпана', '']","['коло', 'подъезд', '']"
3,3,"['но', 'человек', 'не', 'мочь', 'существовать'...",1,"['но', 'человек', 'не', 'мочь', 'существовать'...","['очий', 'люд', 'ударить', 'артём', '']","['н', 'к', 'она', 'приставать', '']"
4,4,"['но', 'человек', 'не', 'мочь', 'существовать'...",0,"['но', 'человек', 'не', 'мочь', 'существовать'...","['очий', 'люд', 'ударить', 'артём', '']","['н', 'она', 'оскорбить', '']"
...,...,...,...,...,...,...
11945,11945,"['известный', 'американский', 'боец', 'смешанн...",0,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['н', 'сочинить', 'гимн', 'советский', 'союз',..."
11946,11946,"['известный', 'американский', 'боец', 'смешанн...",1,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['', 'он', 'тат', 'на', 'русский', 'язык', 'и'..."
11947,11947,"['известный', 'американский', 'боец', 'смешанн...",0,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['а', 'он', 'тело', 'есть', 'татуировка', 'с',..."
11948,11948,"['известный', 'американский', 'боец', 'смешанн...",0,"['известный', 'американский', 'боец', 'смешанн...","['ак', 'проявляться', 'любовь', 'известный', '...","['н', 'писать', 'стих', 'и', 'проза', 'на', 'р..."


# Новый раздел

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
X=train_df.drop(['label'],axis=1)
y=train_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

vectorizer = CountVectorizer(ngram_range=(3,6), analyzer='char', max_features = 25000)

X_train_vector = vectorizer.fit_transform(X_train['text']).toarray()


In [ ]:
X_train_vector1 = vectorizer.transform(X_train.text).toarray()
#X_train_vector2 = vectorizer.transform(X_train.Query).toarray()
#X_train_vector3 = vectorizer.transform(X_train.Answer).toarray()

In [ ]:
X_test_vector1 = vectorizer.transform(X_test.text).toarray()
#X_test_vector2 = vectorizer.transform(X_test.Query).toarray()
#X_test_vector3 = vectorizer.transform(X_test.Answer).toarray()

In [ ]:
type(X_test.text[0])

str

In [ ]:
X_test.text

10390    ['с', 'глаз', 'полный', 'слеза', 'иван', 'гляд...
8888     ['в', 'москва', 'вечером', 'в', 'воскресение',...
3952     ['вечером', 'сид', 'начинать', 'воровать', 'ку...
360      ['на', 'земля', 'быть', 'неуютный', 'и', 'стра...
3781     ['мэрия', 'москва', 'пообещать', 'рассмотреть'...
                               ...                        
10715    ['пользователь', '', 'под', 'ник', '', 'обрати...
940      ['президент', 'сбербанк', 'герман', 'греф', 'п...
4281     ['фигурантка', 'дело', 'оборонсервис', 'евгени...
5553     ['профессиональный', 'гимназия', 'электротехни...
4770     ['в', 'армения', 'арестовать', 'сын', 'губерна...
Name: text, Length: 3944, dtype: object

In [ ]:
for i in range(0,len(test_df.text)):
  test_df.text[i]=str(test_df.text[i])

In [ ]:
test_df.text

0       ['самый', 'первый', 'остров', 'архипелаг', 'во...
1       ['самый', 'первый', 'остров', 'архипелаг', 'во...
2       ['самый', 'первый', 'остров', 'архипелаг', 'во...
3       ['самый', 'первый', 'остров', 'архипелаг', 'во...
4       ['самый', 'первый', 'остров', 'архипелаг', 'во...
                              ...                        
2230    ['самый', 'первый', 'остров', 'архипелаг', 'во...
2231    ['самый', 'первый', 'остров', 'архипелаг', 'во...
2232    ['самый', 'первый', 'остров', 'архипелаг', 'во...
2233    ['самый', 'первый', 'остров', 'архипелаг', 'во...
2234    ['самый', 'первый', 'остров', 'архипелаг', 'во...
Name: text, Length: 2235, dtype: object

In [ ]:
X=vectorizer.transform(test_df.text).toarray()

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 39 kB/s 


In [ ]:
%%time
from sklearn.metrics import classification_report #building a price prediction
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

model = CatBoostClassifier(silent = True)

model.fit(X_train_vector1, y_train)
y_pred_test = model.predict(X_test_vector1)

print(accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

0.5068458417849898
              precision    recall  f1-score   support

           0       0.53      0.78      0.63      2127
           1       0.42      0.18      0.26      1817

    accuracy                           0.51      3944
   macro avg       0.47      0.48      0.44      3944
weighted avg       0.48      0.51      0.46      3944

CPU times: user 16min 23s, sys: 5.02 s, total: 16min 28s
Wall time: 9min 52s


In [ ]:
y_pr=model.predict(X)

In [ ]:
test_pred = pd.DataFrame(y_pr, columns=['label'])
test_pred.reset_index(inplace=True)

#test_pred['label'] = test_pred['label'].map(inverse_dict).fillna(test_pred['lang_samp'])
test_pred

,index,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2230,2230,0
2231,2231,0
2232,2232,0
2233,2233,0


In [ ]:
test_pred.to_csv('submition_tr.csv', index=False)

In [ ]:
from google.colab import files
files.download("submition_tr.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>